# EEGNet Classification Benchmarks (Colab)

Runs EEGNet on the classification datasets you have ready: EEGMAT, BCI-IV-2A, SHU, Siena, and HMC.


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)


In [ ]:
import os
import subprocess

REPO_ROOT = "/content/AdaBrain-Bench"
if not os.path.exists(REPO_ROOT):
    subprocess.run([
        "git", "clone", "--depth", "1",
        "https://github.com/iroblesrazzaq/AdaBrain-Bench.git", REPO_ROOT
    ], check=True)
os.chdir(REPO_ROOT)
subprocess.run(["git", "pull"], check=True)
subprocess.run(["git", "log", "-1"], check=True)


In [ ]:
DATA_ROOT = "/content/drive/MyDrive/AdaBrain-Bench Data"
PREPROCESSING_ROOT = f"{DATA_ROOT}/preprocessing"
print("DATA_ROOT:", DATA_ROOT)
print("PREPROCESSING_ROOT:", PREPROCESSING_ROOT)


In [ ]:
import json

config = {
  "EEGMAT": {
    "root": {
      "cross": f"{PREPROCESSING_ROOT}/EEGMAT/cross_subject_json"
    },
    "num_classes": 2,
    "num_t": 4
  },
  "BCI-IV-2A": {
    "root": {
      "multi": f"{PREPROCESSING_ROOT}/BCI-IV-2A/multi_subject_json",
      "cross": f"{PREPROCESSING_ROOT}/BCI-IV-2A/cross_subject_json",
      "fewshot": f"{PREPROCESSING_ROOT}/BCI-IV-2A/multi_subject_json"
    },
    "num_classes": 4,
    "num_t": 4
  },
  "SHU": {
    "root": {
      "multi": f"{PREPROCESSING_ROOT}/SHU/multi_subject_json",
      "cross": f"{PREPROCESSING_ROOT}/SHU/cross_subject_json",
      "fewshot": f"{PREPROCESSING_ROOT}/SHU/multi_subject_json"
    },
    "num_classes": 2,
    "num_t": 4
  },
  "Siena": {
    "root": {
      "cross": f"{PREPROCESSING_ROOT}/Siena/cross_subject_json"
    },
    "num_classes": 2,
    "num_t": 10
  },
  "HMC": {
    "root": {
      "cross": f"{PREPROCESSING_ROOT}/HMC/cross_subject_json"
    },
    "num_classes": 5,
    "num_t": 30
  }
}

config_path = os.path.join(REPO_ROOT, "dataset_config", "Classification.json")
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)
print("Wrote", config_path)


## Run EEGNet

EEGNet uses a 200 Hz time base in `run_finetuning.py`, so the runs below resample all datasets to 200 Hz.


In [ ]:
DATASETS = ["EEGMAT", "BCI-IV-2A", "SHU", "Siena", "HMC"]

BASE_ARGS = {
    "epochs": 50,
    "batch_size": 64,
    "sampling_rate": 200,
    "num_workers": 4,
    "norm_method": "z_score",
    "seed": 0,
}

DATASET_OVERRIDES = {
    "Siena": {"batch_size": 32},
    "HMC": {"batch_size": 32}
}

DRY_RUN = False


In [ ]:
import sys
import subprocess

for dataset in DATASETS:
    overrides = DATASET_OVERRIDES.get(dataset, {})
    epochs = overrides.get("epochs", BASE_ARGS["epochs"])
    batch_size = overrides.get("batch_size", BASE_ARGS["batch_size"])
    sampling_rate = overrides.get("sampling_rate", BASE_ARGS["sampling_rate"])

    cmd = [
        sys.executable, "run_finetuning.py",
        "--model_name", "EEGNet",
        "--dataset", dataset,
        "--task_mod", "Classification",
        "--subject_mod", "cross",
        "--finetune_mod", "full",
        "--norm_method", BASE_ARGS["norm_method"],
        "--batch_size", str(batch_size),
        "--epochs", str(epochs),
        "--sampling_rate", str(sampling_rate),
        "--seed", str(BASE_ARGS["seed"]),
        "--num_workers", str(BASE_ARGS["num_workers"])
    ]
    if DRY_RUN:
        cmd.append("--dry_run")

    print("\nRunning:", " ".join(cmd))
    subprocess.run(cmd, check=True)


## (Optional) Copy results to Drive

Uncomment and run if you want the results saved back to Drive.


In [ ]:
import os
import subprocess

out_root = os.path.join(REPO_ROOT, "finetuning_results", "Classification", "EEGNet_results")
drive_out = os.path.join(DATA_ROOT, "finetuning_results", "EEGNet")
os.makedirs(drive_out, exist_ok=True)
subprocess.run(["rsync", "-av", out_root + "/", drive_out + "/"], check=True)
print("Copied results to", drive_out)
